# Decorators : closure, variable scope, nonlocal...

---------------------

source : "Fluent Python" by Luciano Ramalho

---------------------

## Decorators 101

Definition : a decorator is a callable that takes another function as argument (the decorated function). The decorator may perform some processing with the decorated function, and return it or replace it with another function or callable object.

Syntax and syntaxer

In [1]:
def decorate(func):
    print('It\'s decorate!')
    return func

In [2]:
@decorate
def target1():
    print('running target1()')
    
target1()

It's decorate!
running target1()


In [3]:
def target2():
    print('running target2()')
    
decorate(target2) # Not decorate(target2())
target2()

It's decorate!
running target2()


-------------

Here is a decorator that replaces a function with a different one.

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [5]:
target()

running inner()


In [6]:
target

<function __main__.deco.<locals>.inner()>

--------------------------------

## When Python executes decorators

![question pour un kahoot](assets/Kahoot_is_not_my_friend_anymore.png)

In [7]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x7f24c8284730>)
running register(<function f2 at 0x7f24c8284950>)
running main()
registry -> [<function f1 at 0x7f24c8284730>, <function f2 at 0x7f24c8284950>]
running f1()
running f2()
running f3()


=> They run right after the decorated function is defined. That is usually at import time.

---------------------

In [9]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_orger(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    return order.total() * .07 if len(distinct_items) >= 10 else 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

--------------

Until now, our decorators don"t change the decorated function. To do so, we usually need to define an inner function and return it to replace the decorated function. Code that uses inner function almost always  depends on closures to operate correctly. And to understand closures, we need to look at how variables scopes work in Python.

----------------

## Variable Scope

In [27]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [28]:
b = 6
f1(3)

3
6


In [29]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

The generate bytecode is different.

In [30]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)
b

3
6


9

---------------

## Closures

A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not define there.

##### An average function using oop

In [10]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [11]:
avg = Averager()
avg(10)

10.0

In [12]:
avg(11)

10.5

In [13]:
avg(12)

11.0

----------------------------

##### Same, but using a higher-order function

In [14]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [15]:
avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

When invoked, make_averager returns an averager function object.

-----------------

Question is : where does this avg function find the series?

---------------

![Closures](assets/closures.svg)

Free variable : a variable that is not bound in the local scope.

In [18]:
avg.__code__.co_varnames

('new_value', 'total')

In [19]:
avg.__code__.co_freevars

('series',)

In [20]:
avg.__closure__

(<cell at 0x7f24cbb01948: list object at 0x7f24c827c948>,)

Each item here corresponds to a name in co_freevars

In [21]:
avg.__closure__[0].cell_contents

[10, 11, 12]

------------------------

###### Conclusion about closure

A closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

Note that the only situation in which a function may need to deal with external variables that are nonglobal is when it's nested in another function.

-------------------------

##### So, let's optimize our code !

In [22]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    
    return averager

In [23]:
avg = make_averager()

In [24]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [25]:
avg(11)

UnboundLocalError: local variable 'count' referenced before assignment

In [26]:
avg(12)

UnboundLocalError: local variable 'count' referenced before assignment

---------------------

## The non local Declaration

nonlocal : lets you flag a variable as a free variable even when it's assigned a new value within the function.

Nonlocal variables are used in nested functions whose local scope is not defined. This means that the variable can be neither in the local nor the global scope.

In [31]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

In [33]:
avg = make_averager()
avg(10)

10.0

In [34]:
avg(11)

10.5

In [35]:
avg(12)

11.0

In [40]:
count

NameError: name 'count' is not defined

----------------

## Implementing a simple decorator

##### A decorator that output the running time of functions

In [47]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [48]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [49]:
print('*' * 25, 'Calling snooze(.125)')
snooze(.125)
print('*' * 25, 'Calling factorial(6)')
factorial(6)

************************* Calling snooze(.125)
0.12521382 snooze(0.125) -> None
************************* Calling factorial(6)
0.00000141 factorial(1) -> 1
0.00008666 factorial(2) -> 2
0.00014799 factorial(3) -> 6
0.00119981 factorial(4) -> 24
0.00127683 factorial(5) -> 120
0.00133028 factorial(6) -> 720


720

- clock gets factorial as its func argument
- It then creates and return the clocked function -> behind the scene, Python assigns it to factorial
- The factorial function is modified.

-----------------

In [50]:
factorial.__name__

'clocked'

Factorial now holds a reference to the clocked function. From now on, each time factorial(n) is called, clocked(n) gets executed.

This is the typical behavior of a decorator. It replaces the decorated function with a new function that accepts the same arguments.

##### But I kinda liked the name of my function without the decorator...

No worries, functools has our back.

In [54]:
import time
import functools

import time

def clock(func):
    @functools.wraps(func)
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [55]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [56]:
factorial.__name__

'factorial'

##### Another decorator in the standard library : functools.lru_cache

Memoization : an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

LRU => Last Recently Used

In [2]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [3]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [4]:
print(fibonacci(6))

0.00000022 fibonacci(0) -> 0
0.00000052 fibonacci(1) -> 1
0.00019762 fibonacci(2) -> 1
0.00000019 fibonacci(1) -> 1
0.00000021 fibonacci(0) -> 0
0.00000015 fibonacci(1) -> 1
0.00010280 fibonacci(2) -> 1
0.00012372 fibonacci(3) -> 2
0.00034122 fibonacci(4) -> 3
0.00000021 fibonacci(1) -> 1
0.00000019 fibonacci(0) -> 0
0.00000055 fibonacci(1) -> 1
0.00022275 fibonacci(2) -> 1
0.00024945 fibonacci(3) -> 2
0.00000017 fibonacci(0) -> 0
0.00000017 fibonacci(1) -> 1
0.00001999 fibonacci(2) -> 1
0.00000014 fibonacci(1) -> 1
0.00000020 fibonacci(0) -> 0
0.00000016 fibonacci(1) -> 1
0.00001947 fibonacci(2) -> 1
0.00003777 fibonacci(3) -> 2
0.00007625 fibonacci(4) -> 3
0.00034463 fibonacci(5) -> 5
0.00076893 fibonacci(6) -> 8
8


fibonacci(1) is called 8 times... => time wasted

In [5]:
import functools

@functools.lru_cache() # can take parameters
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [6]:
print(fibonacci(6))

0.00000021 fibonacci(0) -> 0
0.00000028 fibonacci(1) -> 1
0.00005282 fibonacci(2) -> 1
0.00000086 fibonacci(3) -> 2
0.00029591 fibonacci(4) -> 3
0.00000085 fibonacci(5) -> 5
0.00056368 fibonacci(6) -> 8
8


## Stacked decorators

In [ ]:
@d1
@d2
def f():
    print('f')

is the same as :

In [ ]:
def f():
    print('f')
    
f = d1(d2(f))

## Parameterized Decorators